## 生成选题文档-提示词

```markdown
### input-output
- Folder_input："`D:\Github_lianxh\open2025`"
- filename-input：包含关键词 'outline' 且后缀为 '.md' 的文件
- Folder-output：'{folder_input}\body'

### 任务：

- Input: 读取 {filename-input} 文件。
- Actions：
  - 在 {filename-input} 文档中，以 `# {secTitle}`开头的行为分割线，将该行及以下到下一个一级标题之间的文本写入一个新的 **.md** 文件中
- Output:
  - 存入：{Folder-output} 文件夹
    - 预先清空该文件夹 
  - 文件名为 `{secTitle}.md`
    - 文件后缀为 '.md'，而不是 '.md.md'
  - 屏幕打印：
    - 文件总数
    - 文件名称列表
```


In [1]:
import os
import re
import shutil

# 输入输出路径
folder_input = r"D:\Github_lianxh\open2025"
folder_output = rf"{folder_input}\body"

# 清空输出文件夹
if os.path.exists(folder_output):
    shutil.rmtree(folder_output)
os.makedirs(folder_output)

# 获取符合条件的输入文件名（包含 'outline' 且后缀为 '.md'）
target_files = [f for f in os.listdir(folder_input) 
                if 'outline' in f and f.endswith('.md')]

if not target_files:
    print("未找到包含 'outline' 的 .md 文件")
    exit()

# 默认只处理第一个符合条件的文件
filename_input = os.path.join(folder_input, target_files[0])

# 读取文件内容
with open(filename_input, "r", encoding="utf-8") as file:
    lines = file.readlines()

# 用于保存分段
sections = {}
current_title = None
buffer = []

# 分割逻辑：以 "##" 或 ">##" 开头的二级标题作为起始点
pattern = re.compile(r"#\s+(.*)")

for line in lines:
    match = pattern.match(line)
    if match:
        # 如果遇到新标题，先保存上一段
        if current_title and buffer:
            sections[current_title] = buffer
        # 启动新的 buffer
        current_title = match.group(1).strip()
        buffer = [line]  # 包括当前标题行
    else:
        if current_title:
            buffer.append(line)

# 最后一节也需要保存
if current_title and buffer:
    sections[current_title] = buffer

# 写入输出文件夹
file_list = []
for title, content in sections.items():
    safe_title = re.sub(r'[\\/:*?"<>|]', '_', title)  # 防止文件名非法字符
    output_path = os.path.join(folder_output, f"{safe_title}.md")
    with open(output_path, "w", encoding="utf-8") as f:
        f.writelines(content)
    file_list.append(f"{safe_title}.md")

# 输出统计信息
print(f"共生成 {len(file_list)} 个文件：")
for name in file_list:
    print(f" - {name}")


共生成 8 个文件：
 - 0. 课程概览.md
 - A. 报名和参与.md
 - B. 课程缘起.md
 - T1. Python：安装和环境配置.md
 - T2. 工具变量：从 LATE 困境到 Shift-Share 争议.md
 - T3. 生成式大语言模型在金融会计研究中的应用.md
 - T4. 路径分析指南.md
 - T5. AI 赋能微观调查数据清洗.md
